# In this notebook we are going to analyze data from the Make School Summer Academy

## We are going to try to find out what is the NPS (Net promoter score) of the summer academy.
## For that we need to check how likely someone is to recommand the Summer Academy to a friend.
#### For people with a score of 1-6, we can consider them detractors.
#### For people with a score of 6-8, we can consider them passive.
#### And for people with a score of 8-10, we can consider them promoters.
#### By averaging all of those out we can find the NPS.

### Let's start by importing what we need

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import re
%matplotlib inline

## We are going to work with two datasets: 2016 and 2017

### Since the one from 2017 is cleaner we will get started by importing that one

In [2]:
df_2017 = pd.read_csv('./2017/Student Feedback Surveys-Superview.csv')
df_2017.head()

,ID,Location,Track,Week,Rating (Num),Schedule Pacing
0,134,San Francisco,"Apps, Explorer",Week 1,3,Just right
1,36,Los Angeles,Apps,Week 1,4,A little too fast
2,117,San Francisco,Games,Week 1,4,Way too slow
3,253,NaN,NaN,Week 2,4,A little too fast
4,350,New York City,"Apps, Explorer",Week 1,4,Just right


### The column we care about is the rating so let's check if anything about it is out of the ordinary

In [3]:
df_2017['Rating (Num)'].unique()

array(['3', '4', '5', '6', '7', '8', '9', '10', '0', '1', '2', '#ERROR!'],
      dtype=object)

### Two things stand out to me:
#### 1. Some of the values in the dataset are '#ERROR!'. We should make sure to get rid of those values.
#### 2. Our numbers are stored as String types and we should convert them to Int in order to work with them.

In [4]:
df_2017_curated = df_2017[df_2017['Rating (Num)'] != '#ERROR!']
df_2017_curated['Rating (Num)'] = df_2017_curated['Rating (Num)'].astype(int)
df_2017_curated['Rating (Num)'].mean()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


8.415172413793103

### We should create a function that helps us if someone is a promoter, passive, or a detractor

In [5]:
def calculate_promo_score(data):
    if data <= 6:
        return -1
    if data <= 8:
        return 0
    return 1

### Now let's use that function to create a new column for the promo score

In [6]:
df_2017_curated['promoter_score'] = df_2017_curated['Rating (Num)'].apply(calculate_promo_score)
df_2017_curated.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,ID,Location,Track,Week,Rating (Num),Schedule Pacing,promoter_score
0,134,San Francisco,"Apps, Explorer",Week 1,3,Just right,-1
1,36,Los Angeles,Apps,Week 1,4,A little too fast,-1
2,117,San Francisco,Games,Week 1,4,Way too slow,-1
3,253,NaN,NaN,Week 2,4,A little too fast,-1
4,350,New York City,"Apps, Explorer",Week 1,4,Just right,-1


### Great! Now let's see what the NPS was for all 2017 summer academies

In [7]:
df_2017_curated['promoter_score'].mean()

0.4406896551724138

### 0.44 is not bad, but that's for all the summer academies. I'm sure some have been more successful than others. Let's create a sweet visualization and see how different cities have done

## Now that we saw how the 2017 summer academy has done let's deal with the more messy dataset of the 2016 summer academy

### Let's start by importing all the weekly feedbacks except for Week 8 since that one is in a quite different format

In [8]:
path = r'./2016'
allFiles = glob.glob(path + "/*.csv")
df_2016_weeks1_6 = pd.DataFrame()
list_ = []
for file in allFiles:
    m = re.search('./2016/Anon Week (\d) Feedback - (.*)\.csv', file)
    week = m.group(1)
    location = m.group(2)
    df = pd.read_csv(file,index_col=None, header=0)
    df['Week'] = week
    df['location'] = location
    list_.append(df)
df_2016_weeks1_6 = pd.concat(list_)
df_2016_weeks1_6.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  del sys.path[0]


,How well are the tutorials paced?,How well is the schedule paced?,How would you rate your overall satisfaction with the Summer Academy this week?,Timestamp,Unnamed: 0,Week,What track are you in?,location
0,NaN,3,3,8/5/2016 1:39:41,NaN,7,NaN,Taipei
1,NaN,3,4,8/5/2016 1:40:47,NaN,7,NaN,Taipei
2,NaN,3,4,8/5/2016 1:40:50,NaN,7,NaN,Taipei
3,NaN,4,4,8/5/2016 1:42:44,NaN,7,NaN,Taipei
4,NaN,4,5,8/5/2016 1:45:13,NaN,7,NaN,Taipei


### Taking a look at this data, it has a satisfaction rating, but nothing about how likely people are to recommend it to friends. For the purposes of this project, I am going to assume that they are the same, but keep in mind, the data might be slightly obscured.
### Let's rename the rating collumn to have a simpler name and check out any unique values it may have.

In [9]:
COLUMNS = {'How would you rate your overall satisfaction with the Summer Academy this week?': 'Rating (Num)'}

df_2016_weeks1_6.rename(index=str, columns=COLUMNS, inplace=True)
df_2016_weeks1_6['Rating (Num)'].unique()

array([3, 4, 5, 2, 1, '5', '4', '3', '#REF!'], dtype=object)

### We can see that, just like the 2017 df, this has an '@REF!' entry and not all ratings are stored as inegers. In addition it is on a scale from 1-5 when we wanna use 1-10 so let's also convert it to the right scale.

In [10]:

df_2016_weeks1_6_curated = df_2016_weeks1_6[df_2016_weeks1_6['Rating (Num)'] != '#REF!']
df_2016_weeks1_6_curated['Rating (Num)'] = (df_2016_weeks1_6_curated['Rating (Num)'].astype(int) * 2)
df_2016_weeks1_6_curated['Rating (Num)'].mean()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


8.38012958963283


### Before we calculate the NPS, let's throw the data from week 8 in the equation.

In [11]:
df_2016_week8 = pd.read_csv('Week 8 Feedback (2016, incomplete) - results.csv')
df_2016_week8['Week'] = 8
df_2016_week8.head()

,#,How likely is it that you would recommend the Make School Summer Academy to a friend?,location,track,Start Date (UTC),Submit Date (UTC),Network ID,Week
0,00b836bda84e6bdbe780af97e249e59f,10,New York,summerApps,9/7/16 1:03,9/7/16 1:04,3212b7a834,8
1,39dde6dc0e1e375845d756fc7e39fc5f,10,San Francisco,summerIntro,9/7/16 1:03,9/7/16 1:04,f4954355aa,8
2,5e56b9de91670b308cb98dd2848b8739,10,New York,summerIntro,9/7/16 1:03,9/7/16 1:05,3d69ca289b,8
3,641081d05785b47a0f17448625da0d49,9,Sunnyvale,summerApps (4-week),9/7/16 1:04,9/7/16 1:06,261608f95d,8
4,c29bdd4f5678d78b450f4494e0f53c8c,3,San Francisco,summerIntro,9/7/16 1:04,9/7/16 1:11,d6672ddf6f,8


### This looks promising! It has the exact data we're looking for! Let's rename the rating colum to something more simple and check if there's anything weird about it.

In [12]:
COLUMNS = {'How likely is it that you would recommend the Make School Summer Academy to a friend?': 'Rating (Num)'}

df_2016_week8.rename(index=str, columns=COLUMNS, inplace=True)
df_2016_week8['Rating (Num)'].unique()

array([10,  9,  3,  8,  6,  7,  4,  5])

### Now let's use all the data from 2016 to create one massive 2016 dataframe!
### We should start by comparing the two DFs side by side and seeing what they have in common

In [13]:
df_2016_week8.head()

,#,Rating (Num),location,track,Start Date (UTC),Submit Date (UTC),Network ID,Week
0,00b836bda84e6bdbe780af97e249e59f,10,New York,summerApps,9/7/16 1:03,9/7/16 1:04,3212b7a834,8
1,39dde6dc0e1e375845d756fc7e39fc5f,10,San Francisco,summerIntro,9/7/16 1:03,9/7/16 1:04,f4954355aa,8
2,5e56b9de91670b308cb98dd2848b8739,10,New York,summerIntro,9/7/16 1:03,9/7/16 1:05,3d69ca289b,8
3,641081d05785b47a0f17448625da0d49,9,Sunnyvale,summerApps (4-week),9/7/16 1:04,9/7/16 1:06,261608f95d,8
4,c29bdd4f5678d78b450f4494e0f53c8c,3,San Francisco,summerIntro,9/7/16 1:04,9/7/16 1:11,d6672ddf6f,8


In [14]:
df_2016_weeks1_6_curated.head()

,How well are the tutorials paced?,How well is the schedule paced?,Rating (Num),Timestamp,Unnamed: 0,Week,What track are you in?,location
0,NaN,3,6,8/5/2016 1:39:41,NaN,7,NaN,Taipei
1,NaN,3,8,8/5/2016 1:40:47,NaN,7,NaN,Taipei
2,NaN,3,8,8/5/2016 1:40:50,NaN,7,NaN,Taipei
3,NaN,4,8,8/5/2016 1:42:44,NaN,7,NaN,Taipei
4,NaN,4,10,8/5/2016 1:45:13,NaN,7,NaN,Taipei


### It looks like we can change a few more things before we merge them

In [15]:
df_2016_weeks1_6_curated.drop('Unnamed: 0', axis = 1)
df_2016_weeks1_6_curated.rename(index=str, columns={'What track are you in?': 'track'}, inplace=True)
df_2016_weeks1_6_curated.head()

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,How well are the tutorials paced?,How well is the schedule paced?,Rating (Num),Timestamp,Unnamed: 0,Week,track,location
0,NaN,3,6,8/5/2016 1:39:41,NaN,7,NaN,Taipei
1,NaN,3,8,8/5/2016 1:40:47,NaN,7,NaN,Taipei
2,NaN,3,8,8/5/2016 1:40:50,NaN,7,NaN,Taipei
3,NaN,4,8,8/5/2016 1:42:44,NaN,7,NaN,Taipei
4,NaN,4,10,8/5/2016 1:45:13,NaN,7,NaN,Taipei


In [16]:
frames = [df_2016_weeks1_6_curated, df_2016_week8]
df_2016 = pd.concat(frames)
df_2016.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  


,#,How well are the tutorials paced?,How well is the schedule paced?,Network ID,Rating (Num),Start Date (UTC),Submit Date (UTC),Timestamp,Unnamed: 0,Week,location,track
0,NaN,NaN,3,NaN,6,NaN,NaN,8/5/2016 1:39:41,NaN,7,Taipei,NaN
1,NaN,NaN,3,NaN,8,NaN,NaN,8/5/2016 1:40:47,NaN,7,Taipei,NaN
2,NaN,NaN,3,NaN,8,NaN,NaN,8/5/2016 1:40:50,NaN,7,Taipei,NaN
3,NaN,NaN,4,NaN,8,NaN,NaN,8/5/2016 1:42:44,NaN,7,Taipei,NaN
4,NaN,NaN,4,NaN,10,NaN,NaN,8/5/2016 1:45:13,NaN,7,Taipei,NaN


### Now that we have everything we need in once place let's calculate our NPS for 2016!
### Conveniently, we have already defined a method for this when working with the 2017 df.

In [17]:
df_2016['promoter_score'] = df_2016['Rating (Num)'].apply(calculate_promo_score)
df_2016.head()

,#,How well are the tutorials paced?,How well is the schedule paced?,Network ID,Rating (Num),Start Date (UTC),Submit Date (UTC),Timestamp,Unnamed: 0,Week,location,track,promoter_score
0,NaN,NaN,3,NaN,6,NaN,NaN,8/5/2016 1:39:41,NaN,7,Taipei,NaN,-1
1,NaN,NaN,3,NaN,8,NaN,NaN,8/5/2016 1:40:47,NaN,7,Taipei,NaN,0
2,NaN,NaN,3,NaN,8,NaN,NaN,8/5/2016 1:40:50,NaN,7,Taipei,NaN,0
3,NaN,NaN,4,NaN,8,NaN,NaN,8/5/2016 1:42:44,NaN,7,Taipei,NaN,0
4,NaN,NaN,4,NaN,10,NaN,NaN,8/5/2016 1:45:13,NaN,7,Taipei,NaN,1


### Now that we can, let's find out what the NPS was for the whole of the summer academy

In [18]:
df_2016['promoter_score'].mean()

0.2462159434914228

## 0.25?! That's not too good!
### Keep in mind that we added the satisfaction rating to this equation because we have considered it good enough. Although our data might be accurate enough we should check it against week 8 just to make sure (Week 8 little data available, but it's the only week that has real data on recommandation scores)

In [19]:
df_2016_week8['promoter_score'] = df_2016_week8['Rating (Num)'].apply(calculate_promo_score)
df_2016_week8['promoter_score'].mean()

0.5538461538461539

## Some interesting findings. 2017 is simple so we can easily tell that the NPS is 0.44
.
## For 2016, however, it is a more complicated situation. While the Week 8 data might be the only one that gives us exactly what we want, we have a small sample with only tens of feedbacks submited. In this case we get a NPS of 0.25.
## We can combine this data with the not-so-accurate data of the "overall satisfaction" field, which we have way more of, but is arguably less accurate for our specific situation. In this case the NPS is 0.55.
## Which one do you think is better and why?